# 파트 06
----
아파트 매매와 인구이동 데이터의 비교분석을 용이하게 하기 위한 전처리 과정을 행한다.

우선적으로 년월의 구분 방법이 다른 것을 같은 형태로 만든다. 또한 데이터 조작의 편의를 위해서 칼럼 이름을 영문으로 전부 고치고 비교 가능한 칼럼들은 같은 이름으로 통일한다.

이후 대부분의 코드는 행정코드의 불일치를 고치는 과정이다. 행정코드의 불일치는 크게 두가지 이유로 남아있다.

하나는 아파트 매매의 경우 주소를 무조건적으로 처음 두단어로 축약하여 코드로 매핑을 하였지만 인구 이동의 경우 행정코드 앞 다섯자리만 남기는 방식으로 전처리를 거쳤기 때문에 방식의 차이로 인해 코드 불일치가 일어났다. 일반적으로 인구 이동 데이터에서의 행정코드가 더 세밀한 단위까지 구분되어 있다. 이를 해결하기 위해 인구 이동 데이터에서 행정코드 -> 주소 -> 축약주소 -> 행정코드 과정을 거쳐 최대한 아파트 매매의 행정코드와 일치시킨다.

나머지 하나는 행정구역의 변화로 인하여 두개의 데이터 프레임은 같은 지역임에도 서로 다른 행정코드를 가지고 있는 경우도 있으며 다행히 이 경우는 약 10가지 경우 남짓하여 손으로 직접 수정을 가한다.

In [1]:
%matplotlib inline
import re
from __future__ import division
from __future__ import print_function

In [2]:
# setting the folder path and read in data
path = 'raw_data\\'
df_pop = pd.read_csv(path + '05_pop_export.csv', encoding='utf-8')
df_estate = pd.read_csv(path + '02_extate_export.csv', encoding='utf-8')

In [3]:
pop_column_dic = {
    u'전입년': 'year',
    u'전입월': 'month',
    u'행정코드': 'area_code',
    u'전입': 'move_in',
    u'타지전입': 'foreign',
    u'사유': 'reason',
    u'총세대': 'house_total',
    u'세대주': 'head',
    u'세대주_남': 'head_male',
    u'세대주_나이': 'head_age',
    u'다인세대': 'multi',
    u'이동_총인구': 'moved_ppl',
    u'이동_남인구': 'moved_male',
}

estate_column_dic = {
    u'시군구': 'address',
    u'전용면적(㎡)': 'area',
    u'년': 'year',
    u'거래금액(만원)': 'price',
    u'건축년도': 'built',
    u'행자부코드': 'area_code',
    u'월': 'month',
}

In [4]:
estate_column_order = [
    'year', 'month', 'area_code', 'area', 'price', 'built', 'address'
]

In [5]:
df_pop.head()

,전입년,전입월,행정코드,전입,타지전입,사유,총세대,세대주,세대주_남,세대주_나이,다인세대,이동_총인구,이동_남인구
0,2006,1,11110,1,0,1,309,258,178,42.344961,121,588,326
1,2006,1,11110,1,0,2,32,8,6,48.500000,8,50,23
2,2006,1,11110,1,0,3,134,129,87,41.945736,59,264,135
3,2006,1,11110,1,0,4,12,11,3,38.000000,8,26,8
4,2006,1,11110,1,0,5,2,2,1,30.500000,0,2,1


In [6]:
df_estate.head()

,시군구,전용면적(㎡),거래금액(만원),층,건축년도,년,월,행자부코드
0,세종특별자치시 금남면,43.01,4520,4,2000,2006,1,36110
1,세종특별자치시 금남면,43.01,4495,10,2000,2006,1,36110
2,세종특별자치시 금남면,43.01,4495,5,2000,2006,1,36110
3,세종특별자치시 금남면,43.01,4520,2,2000,2006,1,36110
4,세종특별자치시 금남면,43.01,4520,5,2000,2006,1,36110


In [7]:
df_pop.rename(columns=pop_column_dic, inplace=True)
df_estate.rename(columns=estate_column_dic, inplace=True)
df_estate = df_estate[estate_column_order]

In [8]:
df_pop.head()

,year,month,area_code,move_in,foreign,reason,house_total,head,head_male,head_age,multi,moved_ppl,moved_male
0,2006,1,11110,1,0,1,309,258,178,42.344961,121,588,326
1,2006,1,11110,1,0,2,32,8,6,48.500000,8,50,23
2,2006,1,11110,1,0,3,134,129,87,41.945736,59,264,135
3,2006,1,11110,1,0,4,12,11,3,38.000000,8,26,8
4,2006,1,11110,1,0,5,2,2,1,30.500000,0,2,1


In [9]:
df_estate.head()

,year,month,area_code,area,price,floor,built,address
0,2006,1,36110,43.01,4520,4,2000,세종특별자치시 금남면
1,2006,1,36110,43.01,4495,10,2000,세종특별자치시 금남면
2,2006,1,36110,43.01,4495,5,2000,세종특별자치시 금남면
3,2006,1,36110,43.01,4520,2,2000,세종특별자치시 금남면
4,2006,1,36110,43.01,4520,5,2000,세종특별자치시 금남면


In [10]:
# we see that the number of area codes do not match up between the dataframes
df_pop.groupby('area_code').count().shape

(264, 12)

In [11]:
df_estate.groupby('area_code').count().shape

(228, 7)

In [12]:
# we try to see the difference in the area codes used in both dataframes
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([41280, 41190, 41130, 41110, 41270, 41170, 41460, 47110, 45110, 43110], dtype=int64)

In [13]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([28720, 41111, 41113, 41115, 41117, 41131, 41133, 41135, 41171,
       41173, 41195, 41197, 41199, 41271, 41273, 41281, 41285, 41287,
       41461, 41463, 41465, 41730, 43111, 43113, 43710, 44730, 44830,
       45111, 45113, 47111, 47113, 48160, 48190, 49110, 49130, 49710,
       49720, 44131, 44133, 48121, 48123, 48125, 48127, 48129, 43112, 43114], dtype=int64)

In [14]:
# just a custom function to use as an aggregate method
def agg_nunique(series):
    return series.nunique()

In [15]:
# we check to see if each address correspond to a unique area code
df_temp = df_estate.groupby('area_code').agg(agg_nunique)
df_temp.loc[df_temp.address != 1]

,year,month,area,price,floor,built,address
area_code,,,,,,,
36110,10,12,234.0,1165,30,23,10


In [16]:
# we do this just in case
# this should give an empty dataframe due to construction
df_temp = df_estate.groupby('address').agg(agg_nunique)
df_temp.loc[df_temp.area_code != 1]

,year,month,area_code,area,price,floor,built
address,,,,,,,


In [17]:
# read in address code excel to see what the problem is
df_address_code = pd.read_excel(path + 'address_code.xlsx', header=2)

In [18]:
# function to shrink multiple white spaces into a single space
def shrink_spaces(address):
    return re.sub(u'\s+', u' ', address)

# function that returns the first two words of the string
# we have modified the function from the previous version to
# accomodate single word addresses
subaddress_regex = re.compile(u'\S+\s+\S+')
def shrink_address(address):
    temp = subaddress_regex.search(address)
    if temp:
        return address[:temp.end()]
    else:
        return address

In [19]:
# tidying up address code dataframe
df_address_code[u'행정구역명'] = df_address_code[u'행정구역명'].apply(shrink_spaces)
df_address_code[u'행자부코드'] = df_address_code[u'행자부코드'] // 100000

In [20]:
# just rehashing previously used code
# we create two series this time as we will map area codes to long form addresses,
# shrink the long form addresses to short form (one or two words),
# and then map back to area codes
sr_address_to_code = pd.Series(data=df_address_code.set_index(u'행정구역명')[u'행자부코드'], copy=True)
sr_address_to_code = sr_address_to_code[~sr_address_to_code.index.duplicated(keep='first')]

sr_code_to_address = pd.Series(data=df_address_code.set_index(u'행자부코드')[u'행정구역명'], copy=True)
sr_code_to_address = sr_code_to_address[~sr_code_to_address.index.duplicated(keep='last')]

In [21]:
pd.DataFrame(data={
    'area_code': estate_area_code_missing.values,
    'address': estate_area_code_missing.map(sr_code_to_address).apply(shrink_address)
})

,address,area_code
695,경기도 고양시,41280
2930,경기도 부천시,41190
3214,경기도 성남시,41130
4091,경기도 수원시,41110
5208,경기도 안산시,41270
5635,경기도 안양시,41170
7097,경기도 용인시,41460
11666,경상북도 포항시,47110
23775,전라북도 전주시,45110
25205,충청북도 청주시,43110


In [22]:
# comparing with the table above, we see that in many cases, the problem arises from
# the area codes being too granular in the population dataframe
pd.DataFrame(data={
    'area_code': pop_area_code_missing.values,
    'address': pop_area_code_missing.map(sr_code_to_address).apply(shrink_address)
})

,address,area_code
800,인천광역시 옹진군,28720
1019,경기도 수원시,41111
1033,경기도 수원시,41113
1047,경기도 수원시,41115
1061,경기도 수원시,41117
1074,경기도 성남시,41131
1088,경기도 성남시,41133
1102,경기도 성남시,41135
1130,경기도 안양시,41171
1144,경기도 안양시,41173


In [23]:
# run the address code through the process explained above
temp_address = df_pop.area_code.map(sr_code_to_address)
temp_address = temp_address.apply(shrink_spaces)
temp_address = temp_address.apply(shrink_address)
temp_address = temp_address.map(sr_address_to_code)
df_pop.area_code = temp_address

In [24]:
# we repeat the process as before trying to see the changes
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([], dtype=int64)

In [25]:
pd.DataFrame(data={
    'area_code': estate_area_code_missing.values,
    'address': estate_area_code_missing.map(sr_code_to_address).apply(shrink_address)
})

,address,area_code


In [26]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([28720, 41730, 43710, 44730, 44830, 48150, 48190, 49110, 49130,
       49710, 49720], dtype=int64)

In [27]:
pd.DataFrame(data={
    'area_code': pop_area_code_missing.values,
    'address': pop_area_code_missing.map(sr_code_to_address).apply(shrink_address)
})

,address,area_code
800,인천광역시 옹진군,28720
1561,경기도 여주군,41730
1888,충청북도 청원군,43710
2114,충청남도 연기군,44730
2198,충청남도 당진군,44830
3000,경상남도 마산시,48150
3028,경상남도 진해시,48190
3252,제주도 제주시,49110
3266,제주도 서귀포시,49130
3280,제주도 북제주군,49710


In [28]:
# as there are only 11 values left to fix, we tidy up things manually
# 옹진군 is actually a collection of small islands, hence it is reasonable there are no appartments in the area
# however we make it part of 중구 for ease of use in manipulating data with consistency
# 여주군 has changed to 여주시
# 청원군 has become part of 청주시
# 연기군 has become part of 세종특별자치시
# 당진군 has changed to 당진시
# 마산시, 진해시 have become part of 창원시
# 제주도 has changed to 제주특별자치도

In [29]:
# creating resources to fix area codes
def area_code_fix(x):
    if x in area_code_fix_map.keys():
        return area_code_fix_map[x]
    else:
        return x

In [30]:
area_code_fix_map = {
    28720: 28110, # 용진군: 중구
    41730: 41670, # 여주군: 여주시
    43710: 43110, # 청원군: 청주시
    44730: 36110, # 연기군: 세종특별자치시
    44830: 44270, # 당진군: 당진시
    48150: 48110, # 마산시: 창원시
    48190: 48110, # 진해시: 창원시
    49110: 50110, # 제주시: 제주시
    49130: 50130, # 서귀포시: 서귀포시
    49710: 50110, # 북제주군: 제주시
    49720: 50130, # 남제주군: 서귀포시
}

In [31]:
df_pop.area_code = df_pop.area_code.apply(area_code_fix)

In [32]:
# one last sanity check
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([], dtype=int64)

In [33]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([], dtype=int64)

In [34]:
# we drop the address string to save space as we can recover them later on
del df_estate['address']

In [35]:
# exporting scrubbed dataframes
df_estate.to_csv(path + '06_estate_export.csv', index=False, encoding='utf-8')

In [36]:
df_pop.to_csv(path + '06_pop_export.csv', index=False, encoding='utf-8')

In [37]:
# we export a series that can easily be used to convert area codes to address strings
sr_code_to_address.apply(shrink_address).to_csv(path + 'code_to_address.csv', encoding='utf-8')